# GPPR (Greedy Proportional-Priority Reduction) implementation

### Import Statements

In [1]:
import time
from typing import List, Tuple, Dict

### Helper Functions

In [2]:
def compute_basic_metrics(s, m, b, r):
    """
    Compute core metrics for a single budget vector.

    s, m, b : lists of numbers (same length)
    r       : savings rate (0.0 to 1.0)
    """
    n = len(s)
    total_spend = sum(s)
    total_budget = sum(b)
    target_budget = (1.0 - r) * total_spend
    required_cut = total_spend - target_budget
    actual_cut = total_spend - total_budget
    target_error = total_budget - target_budget

    min_ok = all(b[i] + 1e-9 >= m[i] for i in range(n))
    target_reached = actual_cut + 1e-9 >= required_cut  # allow tiny float slack

    return {
        "total_spend": total_spend,
        "total_budget": total_budget,
        "target_budget": target_budget,
        "required_cut": required_cut,
        "actual_cut": actual_cut,
        "target_error": target_error,
        "min_ok": min_ok,
        "target_reached": target_reached,
    }

In [5]:
def print_budget_table(categories, s, m, p, b):
    """
    Nicely print a table of old vs new budgets for human reading.
    """
    n = len(s)
    if categories is None or len(categories) != n:
        categories = [f"C{i+1}" for i in range(n)]

    header = (
        f"{'Category':<15} {'Spend s[i]':>10} {'Min m[i]':>10} "
        f"{'Priority':>10} {'Budget b[i]':>12} {'Cut (s-b)':>12}"
    )
    print(header)
    print("-" * len(header))

    for i in range(n):
        cut = s[i] - b[i]
        print(
            f"{categories[i]:<15} "
            f"{s[i]:>10.2f} {m[i]:>10.2f} {p[i]:>10d} "
            f"{b[i]:>12.2f} {cut:>12.2f}"
        )

### Algorithm Implementation

In [4]:
def gppr_budget(
    s: List[float],
    m: List[float],
    p: List[int],
    r: float,
    adjustment_unit: float = 10.0,
) -> Tuple[List[float], Dict]:
    """
    Greedy Proportional-Priority Reduction (GPPR) algorithm.

    Inputs
    ------
    s : list of last-month spends per category
    m : list of minimum acceptable budgets per category
    p : list of priorities (1 = highest protection, larger numbers = lower priority)
    r : global savings rate, e.g., 0.15 for 15%
    adjustment_unit : amount (same units as s) used in the fine-tuning loop

    Returns
    -------
    b : list of new budgets per category
    diagnostics : dict with metrics and internal stats
    """
    start_time = time.time()
    n = len(s)
    assert len(m) == n and len(p) == n, "s, m, p must have the same length"

    # --- Step 1: compute totals and required cut ---
    total_spend = sum(s)
    target_budget = (1.0 - r) * total_spend
    required_cut = total_spend - target_budget

    ops_count = 0

    # --- Step 2: trivial case, no cuts needed ---
    if required_cut <= 0:
        b = [max(s[i], m[i]) for i in range(n)]
        ops_count += n

        diagnostics = compute_basic_metrics(s, m, b, r)
        diagnostics.update({
            "algorithm": "GPPR",
            "discretionary_total": 0.0,
            "feasible_full_target": True,
            "ops_count": ops_count,
            "adjustment_iterations": 0,
            "runtime_ms": (time.time() - start_time) * 1000.0,
        })
        return b, diagnostics

    # --- Step 3: compute discretionary spend per category ---
    discretionary = []
    for i in range(n):
        d = max(s[i] - m[i], 0.0)
        discretionary.append(d)
        ops_count += 1

    discretionary_total = sum(discretionary)
    ops_count += n

    # --- Step 4: feasibility check ---
    if discretionary_total <= required_cut + 1e-9:
        # Cannot fully achieve target; cut everything to minimum
        b = [float(m[i]) for i in range(n)]
        ops_count += n

        diagnostics = compute_basic_metrics(s, m, b, r)
        diagnostics.update({
            "algorithm": "GPPR",
            "discretionary_total": discretionary_total,
            "feasible_full_target": False,
            "ops_count": ops_count,
            "adjustment_iterations": 0,
            "runtime_ms": (time.time() - start_time) * 1000.0,
        })
        return b, diagnostics

    # --- Step 5: compute weights based on priority and discretionary spend ---
    weights = [0.0] * n
    weight_sum = 0.0
    for i in range(n):
        if discretionary[i] > 0:
            priority_factor = 1.0 / float(p[i])  # higher priority (1) → larger factor
            w = priority_factor * discretionary[i]
            weights[i] = w
            weight_sum += w
            ops_count += 3  # rough
        else:
            weights[i] = 0.0
            ops_count += 1

    if weight_sum == 0:
        b = [float(m[i]) for i in range(n)]
        ops_count += n

        diagnostics = compute_basic_metrics(s, m, b, r)
        diagnostics.update({
            "algorithm": "GPPR",
            "discretionary_total": discretionary_total,
            "feasible_full_target": False,
            "ops_count": ops_count,
            "adjustment_iterations": 0,
            "runtime_ms": (time.time() - start_time) * 1000.0,
        })
        return b, diagnostics

    # --- Step 6: greedy proportional cuts ---
    proposed_cut = [0.0] * n
    for i in range(n):
        if discretionary[i] <= 0:
            proposed_cut[i] = 0.0
            ops_count += 1
            continue
        raw_cut = required_cut * (weights[i] / weight_sum)
        # Cap by discretionary spend
        c = min(raw_cut, discretionary[i])
        proposed_cut[i] = c
        ops_count += 4  # rough

    # --- Step 7: preliminary budgets and actual cut used ---
    b = [0.0] * n
    actual_cut = 0.0
    for i in range(n):
        # Don't go below m[i]
        bi = max(s[i] - proposed_cut[i], m[i])
        b[i] = bi
        actual_cut += (s[i] - bi)
        ops_count += 4

    remaining_cut = required_cut - actual_cut
    ops_count += 1

    # --- Step 8: adjustment loop ---
    #   if remaining_cut > 0 : we need to cut a bit more
    #   if remaining_cut < 0 : we need to add back some budget
    # For ordering:
    #   - further cuts from LOWER priority (larger p)
    #   - restore budgets to HIGHER priority (smaller p)
    
    indices_by_low_priority = sorted(range(n), key=lambda i: p[i], reverse=True)
    indices_by_high_priority = sorted(range(n), key=lambda i: p[i])

    max_iterations = 100000
    iterations = 0

    if adjustment_unit <= 0:
        adjustment_unit = 1.0

    while abs(remaining_cut) > 1e-6 and iterations < max_iterations:
        iterations += 1
        changed = False

        if remaining_cut > 0:
            for i in indices_by_low_priority:
                room = b[i] - m[i]
                if room <= 1e-9:
                    continue
                delta = min(adjustment_unit, remaining_cut, room)
                if delta <= 1e-9:
                    continue
                b[i] -= delta
                remaining_cut -= delta
                actual_cut += delta
                changed = True
                ops_count += 6

                if abs(remaining_cut) <= 1e-6:
                    break

        else:
            # remaining_cut < 0 → if cut too much, add back to high-priority first
            need_to_add = -remaining_cut
            for i in indices_by_high_priority:
                room = s[i] - b[i]
                if room <= 1e-9:
                    continue
                delta = min(adjustment_unit, need_to_add, room)
                if delta <= 1e-9:
                    continue
                b[i] += delta
                remaining_cut += delta
                actual_cut -= delta
                changed = True
                ops_count += 6

                if abs(remaining_cut) <= 1e-6:
                    break

        if not changed:
            break

    # --- Step 9: finalize diagnostics ---
    basic = compute_basic_metrics(s, m, b, r)
    diagnostics = {
        "algorithm": "GPPR",
        "discretionary_total": discretionary_total,
        "feasible_full_target": discretionary_total > required_cut + 1e-9,
        "ops_count": ops_count,
        "adjustment_iterations": iterations,
        "runtime_ms": (time.time() - start_time) * 1000.0,
    }
    diagnostics.update(basic)

    return b, diagnostics

### Example Demonstration

In [6]:
# Example scenario: simple monthly budget

categories = ["Rent", "Groceries", "Restaurants", "Transport", "Entertainment"]
s = [1200.0, 400.0, 350.0, 150.0, 200.0]   # last month spending
m = [1200.0, 300.0, 100.0, 100.0, 50.0]    # minimum acceptable budgets
p = [1, 2, 4, 3, 5]                        # 1 = highest protection
r = 0.15                                   # want to save 15%

b_gppr, diag_gppr = gppr_budget(s, m, p, r, adjustment_unit=10.0)

print("=== GPPR Budget Result ===")
print_budget_table(categories, s, m, p, b_gppr)
print("\nDiagnostics:")
for k, v in diag_gppr.items():
    print(f"{k:>20}: {v}")

=== GPPR Budget Result ===
Category        Spend s[i]   Min m[i]   Priority  Budget b[i]    Cut (s-b)
--------------------------------------------------------------------------
Rent               1200.00    1200.00          1      1200.00         0.00
Groceries           400.00     300.00          2       300.00       100.00
Restaurants         350.00     100.00          4       214.53       135.47
Transport           150.00     100.00          3       113.87        36.13
Entertainment       200.00      50.00          5       126.60        73.40

Diagnostics:
           algorithm: GPPR
 discretionary_total: 550.0
feasible_full_target: True
           ops_count: 67
adjustment_iterations: 1
          runtime_ms: 0.0
         total_spend: 2300.0
        total_budget: 1955.0
       target_budget: 1955.0
        required_cut: 345.0
          actual_cut: 345.0
        target_error: 0.0
              min_ok: True
      target_reached: True
